# Setup

## Instalação das bibliotecas

In [2]:
# Instalação das bibliotecas pandas, numpy, matplotlib, seaborn
%pip install pandas numpy matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


## Documentação dos arquivos

Documentação do que se trata os arquivos (contexto) e seus dados (colunas)

https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_daily_summary_files

## Criação da estrtutura de diretorios

In [3]:
# Cria a pasta data caso não exista
import os

if not os.path.exists('data'):
    os.makedirs('data')

## Download dos arquivos

Identificação do padrão para download dos arquivos.

- Ozone (44201) - https://aqs.epa.gov/aqsweb/airdata/daily_44201_{ANO}.zip
- SO2 (42401) - https://aqs.epa.gov/aqsweb/airdata/daily_42401_{ANO}.zip
- CO (42101) - https://aqs.epa.gov/aqsweb/airdata/daily_42101_{ANO}.zip
- NO2 (42602) - https://aqs.epa.gov/aqsweb/airdata/daily_42602_{ANO}.zip

Exemplo: https://aqs.epa.gov/aqsweb/airdata/daily_44201_2024.zip

In [4]:
urls_files = []

urlozone_base = 'https://aqs.epa.gov/aqsweb/airdata/daily_44201_{year}.zip',
urlso2_base = 'https://aqs.epa.gov/aqsweb/airdata/daily_42401_{year}.zip',
urlco_base = 'https://aqs.epa.gov/aqsweb/airdata/daily_42101_{year}.zip',
urlno2_base = 'https://aqs.epa.gov/aqsweb/airdata/daily_42602_{year}.zip',

ano_inicio = 2022
ano_fim = 2024

for ano in range(ano_inicio, ano_fim+1):
    urls_files.append(urlozone_base[0].format(year=ano))
    urls_files.append(urlso2_base[0].format(year=ano))
    urls_files.append(urlco_base[0].format(year=ano))
    urls_files.append(urlno2_base[0].format(year=ano))

urls_files


['https://aqs.epa.gov/aqsweb/airdata/daily_44201_2022.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42401_2022.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42101_2022.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42602_2022.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_44201_2023.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42401_2023.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42101_2023.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42602_2023.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_44201_2024.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42401_2024.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42101_2024.zip',
 'https://aqs.epa.gov/aqsweb/airdata/daily_42602_2024.zip']

In [5]:
import urllib.request
import shutil

total_size = 0

for url in urls_files:
    # Faz uma requisição HEAD para obter os cabeçalhos
    with urllib.request.urlopen(urllib.request.Request(url, method='HEAD')) as response:
        total_size += int(response.headers.get('Content-Length', 0))

# Verifica se o tamanho do arquivo é maior que 0
if total_size == 0:
    raise Exception("Os arquivos não existem ou os tamanhos são 0.")

# Verfica se há espaço suficiente no disco
total, used, free = shutil.disk_usage("/")
total_size_MB = round(total_size / (1024 * 1024), 2)

if free < total_size:
    raise Exception(f"Espaço insuficiente no disco. \nDisponível {round(free / (1024 * 1024), 2)} MB. \nNecessário {total_size_MB} MB.")
else:
    print(f"Espaço suficiente no disco. \nDisponível {round(free / (1024 * 1024), 2)} MB. \nNecessário {total_size_MB} MB.")

Espaço suficiente no disco. 
Disponível 975436.91 MB. 
Necessário 28.74 MB.


In [6]:
import zipfile
import os

path = 'data/'

for url in urls_files:
    # Monta o caminho do arquivo
    filepath = path + url.split('/')[-1]

    # Download do arquivo
    urllib.request.urlretrieve(url, filepath)

    # Verifica se o arquivo foi baixado corretamente
    if os.path.exists(filepath):

        # Verifica se o tamanho do arquivo descompactado cabe no disco
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            for file in zip_ref.namelist():
                file_size = zip_ref.getinfo(file).file_size
                if free < file_size:
                    raise Exception(f"Espaço insuficiente no disco para descompactar o arquivo {file}. \nDisponível {round(free / (1024 * 1024), 2)} MB. \nNecessário {round(file_size / (1024 * 1024), 2)} MB.")

            # Descompacta o arquivo
            zip_ref.extractall('data')
    else:
        raise Exception("Erro no download do arquivo.")

# Organização dos arquivos em dicionario

In [23]:
import os

"""
- Ozone (44201) - https://aqs.epa.gov/aqsweb/airdata/daily_44201_{ANO}.zip
- SO2 (42401) - https://aqs.epa.gov/aqsweb/airdata/daily_42401_{ANO}.zip
- CO (42101) - https://aqs.epa.gov/aqsweb/airdata/daily_42101_{ANO}.zip
- NO2 (42602) - https://aqs.epa.gov/aqsweb/airdata/daily_42602_{ANO}.zip
"""

# Pegar lista de arquivos CSVs na pasta data
files = {
    'ozone': [],
    'so2': [],
    'co': [],
    'no2': []
}

for f in os.listdir('data'):
    if f.endswith('.csv'):
        if '44201' in f:
            files['ozone'].append('data/'+f)
        elif '42401' in f:
            files['so2'].append('data/'+f)
        elif '42101' in f:
            files['co'].append('data/'+f)
        elif '42602' in f:
            files['no2'].append('data/'+f)

# Avaliações preliminares da estrutura dos arquivos

### Tamanho do arquivo

Avaliar tamanhos dos arquivos para não gerar estouro de memória e também saber se é possível carregar o dataframe todo de uma vez ou se é necessário fazer o carregamento em partes usando o parâmetro chunksize.

In [8]:
# Tamanho total do arquivo
total_files_size = 0

ozone_size = sum([os.path.getsize(file) for file in files['ozone']])
so2_size = sum([os.path.getsize(file) for file in files['so2']])
co_size = sum([os.path.getsize(file) for file in files['co']])
no2_size = sum([os.path.getsize(file) for file in files['no2']])
total_files_size = ozone_size + so2_size + co_size + no2_size

ozone_size_MB = round(ozone_size / (1024 * 1024), 2)
so2_size_MB = round(so2_size / (1024 * 1024), 2)
co_size_MB = round(co_size / (1024 * 1024), 2)
no2_size_MB = round(no2_size / (1024 * 1024), 2)
total_files_size_MB = round(total_files_size / (1024 * 1024), 2)

print(f"Tamanho do arquivo Ozone: {ozone_size_MB} MB")
print(f"Tamanho do arquivo SO2: {so2_size_MB} MB")
print(f"Tamanho do arquivo CO: {co_size_MB} MB")
print(f"Tamanho do arquivo NO2: {no2_size_MB} MB")
print(f"Tamanho total dos arquivos: {total_files_size_MB} MB")

Tamanho do arquivo Ozone: 335.51 MB
Tamanho do arquivo SO2: 238.25 MB
Tamanho do arquivo CO: 145.33 MB
Tamanho do arquivo NO2: 136.84 MB
Tamanho total dos arquivos: 855.92 MB


### Estrutura (RAW) do arquivo - Cabeçalho, 2 primeiras e 2 últimas linhas

Importante avaliar as colunas do arquivo, os tipos de dados, se as colunas estão completas, e se existem colunas que não são necessárias para a análise.

Ao carregar o dataframe através do pandas, ele já faz uma avaliação inicial dos tipos de dados, mas é importante fazer essa avaliação manual também, pois ele pode inferir tipos de dados errados, já que se baseia nas primeira linhas.

A primeira linha deve ser obrigatoriamente o cabeçalho, e as demais linhas devem ser os dados. Caso o arquivo tenha mais de uma linha de cabeçalho, é necessário usar o parâmetro header=1 ou header=2, por exemplo.

In [9]:
for pollutant, filepath in files.items():

    title = '\nPoluente - ' + pollutant + ':'
    print(title)
    print('-' * len(title))

    with open(filepath[0], 'r') as file:
        # Lê as primeiras 5 e 5 ultimas linhas do arquivo
        lines = file.readlines()

        print("Cabeçalho do arquivo:")
        for line in lines[:1]:
            print(line.strip())

        print("\nPrimeiras 2 linhas do arquivo:")
        for line in lines[1:3]:
            print(line.strip())

        print("\nUltimas 5 linhas do arquivo:")
        for line in lines[-2:]:
            print(line.strip())
        print('\n' + '-'*100)


Poluente - ozone:
------------------
Cabeçalho do arquivo:
"State Code","County Code","Site Num","Parameter Code","POC","Latitude","Longitude","Datum","Parameter Name","Sample Duration","Pollutant Standard","Date Local","Units of Measure","Event Type","Observation Count","Observation Percent","Arithmetic Mean","1st Max Value","1st Max Hour","AQI","Method Code","Method Name","Local Site Name","Address","State Name","County Name","City Name","CBSA Name","Date of Last Change"

Primeiras 2 linhas do arquivo:
"01","003","0010","44201",1,30.497478,-87.880258,"NAD83","Ozone","8-HR RUN AVG BEGIN HOUR","Ozone 8-hour 2015","2024-02-29","Parts per million","None",2,12.0,0.032,0.032,23,30,"047","INSTRUMENTAL - ULTRA VIOLET","FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE,  ALABAMA","Alabama","Baldwin","Fairhope","Daphne-Fairhope-Foley, AL","2024-10-31"
"01","003","0010","44201",1,30.497478,-87.880258,"NAD83","Ozone","8-HR RUN AVG BEGIN HOUR","Ozone 8-hour 2015","2024-03-01","P

# Pandas

## Setup

In [24]:
import pandas as pd

# Configuração do pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)
# Configuração do pandas para exibir todas as linhas
pd.set_option('display.max_rows', None)

## Load Dataframes

In [25]:
"""
- Carregando files usando o low_memory=False para evitar problemas de inferência errada em tipos de dados
- Usando o assign para adicionar o nome do arquivo a uma nova coluna.
- É importante avaliar se os dados da coluna "Date Local" são coerentes com o nome do arquivo, já que os dados dizem ser de determinada data. Caso for diferente, pode indicar inconsistências ou dados incorretos.
"""

# Importar através do link direto e sem extrair
# pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_44201_2024.zip', compression='zip', low_memory=False)

df_ozone = pd.concat([
    pd.read_csv(file, low_memory=False).assign(file_name=os.path.basename(file))
    for file in files['ozone']
])

df_so2 = pd.concat([
    pd.read_csv(file, low_memory=False).assign(file_name=os.path.basename(file))
    for file in files['so2']
])

df_co = pd.concat([
    pd.read_csv(file, low_memory=False).assign(file_name=os.path.basename(file))
    for file in files['co']
])

df_no2 = pd.concat([
    pd.read_csv(file, low_memory=False).assign(file_name=os.path.basename(file))
    for file in files['no2']
])

df_all = pd.concat([df_ozone, df_so2, df_co, df_no2], ignore_index=True)

In [26]:
# Teste de extração do ano do nome do arquivo (daily_44201_2022.csv -> 2022)
df_ozone['file_name'].str.extract(r'_(\d{4})\.csv')[0].head()

0    2024
1    2024
2    2024
3    2024
4    2024
Name: 0, dtype: object

In [27]:
# Extrair o ano do nome do arquivo (daily_44201_2022.csv -> 2022)
df_ozone['file_year'] = df_ozone['file_name'].str.extract(r'_(\d{4})\.csv')[0]
df_so2['file_year'] = df_so2['file_name'].str.extract(r'_(\d{4})\.csv')[0]
df_co['file_year'] = df_co['file_name'].str.extract(r'_(\d{4})\.csv')[0]
df_no2['file_year'] = df_no2['file_name'].str.extract(r'_(\d{4})\.csv')[0]

## Avaliação incial da estrutura do dataframe

### Head / Tail / Sample

Avaliando as 5 primeiras e 5 últimas linhas do dataframe, e também uma amostra aleatória de 10 linhas.

A amostra aleatória é importante para avaliar se os dados estão coerentes e se não há problemas de inconsistências nos dados, ou agrupamentos desbalanceados de dados

In [28]:
pd.concat([df_ozone.head(3), df_ozone.tail(3), df_ozone.sample(5)])

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,file_name,file_year
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-02-29,Parts per million,NaN,2,12.0,0.032000,0.032,23,30.0,47.0,INSTRUMENTAL - ULTRA VIOLET,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31,daily_44201_2024.csv,2024
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-03-01,Parts per million,NaN,34,200.0,0.024706,0.030,12,28.0,47.0,INSTRUMENTAL - ULTRA VIOLET,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31,daily_44201_2024.csv,2024
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-03-02,Parts per million,NaN,34,200.0,0.013118,0.016,18,15.0,47.0,INSTRUMENTAL - ULTRA VIOLET,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31,daily_44201_2024.csv,2024
388235,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-12-29,Parts per million,NaN,17,100.0,0.014647,0.026,9,24.0,87.0,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,Not in a city,NaN,2024-05-25,daily_44201_2023.csv,2023
388236,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-12-30,Parts per million,NaN,17,100.0,0.026000,0.033,11,31.0,87.0,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,Not in a city,NaN,2024-05-25,daily_44201_2023.csv,2023
388237,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-12-31,Parts per million,NaN,12,71.0,0.020917,0.033,10,31.0,87.0,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,Not in a city,NaN,2024-05-25,daily_44201_2023.csv,2023
244872,35,15,1005,44201,1,32.380118,-104.262726,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-06-02,Parts per million,NaN,17,100.0,0.042765,0.055,10,51.0,47.0,INSTRUMENTAL - ULTRA VIOLET,5ZR ON BLM LAND BORDERING RESIDENTIAL AREA OUT...,"2811 HOLLAND ST,CARLSBAD,NM",New Mexico,Eddy,Carlsbad,"Carlsbad-Artesia, NM",2024-05-25,daily_44201_2023.csv,2023
333916,48,201,417,44201,1,29.772915,-95.395784,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-03-19,Parts per million,NaN,17,100.0,0.028294,0.047,10,44.0,47.0,INSTRUMENTAL - ULTRA VIOLET,Houston Harvard Street,160 Harvard Street,Texas,Harris,Houston,"Houston-The Woodlands-Sugar Land, TX",2024-05-25,daily_44201_2023.csv,2023
121530,27,5,2013,44201,1,46.851811,-95.846272,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-06-16,Parts per million,NaN,17,100.0,0.032529,0.045,12,42.0,87.0,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,FWS Wetland Management District,26624 NORTH TOWER ROAD,Minnesota,Becker,Detroit Lakes,NaN,2024-09-24,daily_44201_2024.csv,2024
169985,22,15,8,44201,2,32.536273,-93.748940,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2023-07-26,Parts per million,NaN,17,100.0,0.040471,0.051,10,47.0,87.0,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Shreveport / Airport,1425 Airport Drive,Louisiana,Bossier,Shreveport,"Shreveport-Bossier City, LA",2024-05-25,daily_44201_2023.csv,2023


In [29]:
pd.concat([df_co.head(3), df_co.tail(3), df_co.sample(5)])

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,file_name,file_year
0,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1 HOUR,CO 1-hour 1971,2023-01-01,Parts per million,NaN,24,100.0,0.154167,0.400,23,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-04-09,daily_42101_2023.csv,2023
1,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1 HOUR,CO 1-hour 1971,2023-01-02,Parts per million,NaN,24,100.0,0.120833,0.200,0,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-04-09,daily_42101_2023.csv,2023
2,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1 HOUR,CO 1-hour 1971,2023-01-03,Parts per million,NaN,24,100.0,0.116667,0.200,12,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-04-09,daily_42101_2023.csv,2023
89790,72,113,4,42101,1,18.009558,-66.627249,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2024-06-29,Parts per million,NaN,24,100.0,0.237500,0.300,0,3.0,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,RESIDENTIAL AREA,CIVIL DEFENSEBLDG. SAN ANTONIO URB. PONC,Puerto Rico,Ponce,Ponce,"Ponce, PR",2024-08-19,daily_42101_2024.csv,2024
89791,72,113,4,42101,1,18.009558,-66.627249,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2024-06-30,Parts per million,NaN,24,100.0,0.200000,0.200,0,2.0,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,RESIDENTIAL AREA,CIVIL DEFENSEBLDG. SAN ANTONIO URB. PONC,Puerto Rico,Ponce,Ponce,"Ponce, PR",2024-08-19,daily_42101_2024.csv,2024
89792,72,113,4,42101,1,18.009558,-66.627249,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2024-07-01,Parts per million,NaN,2,8.0,0.200000,0.200,0,2.0,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,RESIDENTIAL AREA,CIVIL DEFENSEBLDG. SAN ANTONIO URB. PONC,Puerto Rico,Ponce,Ponce,"Ponce, PR",2024-08-19,daily_42101_2024.csv,2024
132593,47,9,101,42101,1,35.633480,-83.941606,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2023-08-11,Parts per million,NaN,24,100.0,0.195833,0.200,0,2.0,554.0,INSTRUMENTAL - Gas Filter Correlation Thermo E...,Great Smoky Mountains NP - Look Rock,GREAT SMOKY MOUNTAINS NP LOOK ROCK,Tennessee,Blount,Not in a city,"Knoxville, TN",2024-05-09,daily_42101_2023.csv,2023
77109,25,25,44,42101,1,42.325186,-71.056061,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2023-03-14,Parts per million,NaN,24,100.0,0.200000,0.200,0,2.0,593.0,INSTRUMENTAL - Gas Filter Correlation Teledyne...,VON HILLERN ST,19 VON HILLERN ST,Massachusetts,Suffolk,Boston,"Boston-Cambridge-Newton, MA-NH",2024-05-09,daily_42101_2023.csv,2023
107001,34,17,1002,42101,1,40.731645,-74.066308,NAD83,Carbon monoxide,1 HOUR,CO 1-hour 1971,2022-04-17,Parts per million,NaN,23,96.0,0.034783,0.100,0,NaN,54.0,INSTRUMENTAL - NONDISPERSIVE INFRARED,Jersey City,2828 John F. Kennedy Boulevard,New Jersey,Hudson,Jersey City,"New York-Newark-Jersey City, NY-NJ-PA",2023-03-06,daily_42101_2022.csv,2022
54351,11,1,51,42101,1,38.894770,-76.953426,NAD83,Carbon monoxide,8-HR RUN AVG END HOUR,CO 8-hour 1971,2023-09-02,Parts per million,NaN,24,100.0,0.383333,0.600,1,7.0,593.0,INSTRUMENTAL - Gas Filter Correlation Teledyne...,Near Road,3600 Benning Road N.E.,District Of Columbia,District of Columbia,Washington,"Washington-Arlington-Alexandria, DC-VA-MD-WV",2024-05-09,daily_42101_2023.csv,2023


In [30]:
pd.concat([df_no2.head(3), df_no2.tail(3), df_no2.sample(5)])

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,file_name,file_year
0,1,73,23,42602,1,33.553056,-86.815000,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2022-01-01,Parts per billion,NaN,24,100.0,1.308333,1.9,12,1,200,Teledyne-API Model 200EUP or T200UP - Photolyt...,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-05-22,daily_42602_2022.csv,2022
1,1,73,23,42602,1,33.553056,-86.815000,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2022-01-02,Parts per billion,NaN,24,100.0,1.954167,4.2,9,4,200,Teledyne-API Model 200EUP or T200UP - Photolyt...,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-05-22,daily_42602_2022.csv,2022
2,1,73,23,42602,1,33.553056,-86.815000,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2022-01-03,Parts per billion,NaN,10,42.0,1.530000,2.2,0,2,200,Teledyne-API Model 200EUP or T200UP - Photolyt...,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2024-05-22,daily_42602_2022.csv,2022
156363,72,25,7,42602,1,18.198712,-66.052237,NAD83,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-03-05,Parts per billion,NaN,24,100.0,3.266667,9.0,20,8,599,Instrumental - Chemiluminescence Teledyne API ...,NaN,Luis A. Ferre Highway,Puerto Rico,Caguas,Caguas,"San Juan-Carolina-Caguas, PR",2024-05-22,daily_42602_2023.csv,2023
156364,72,25,7,42602,1,18.198712,-66.052237,NAD83,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-03-06,Parts per billion,NaN,24,100.0,4.516667,8.8,10,8,599,Instrumental - Chemiluminescence Teledyne API ...,NaN,Luis A. Ferre Highway,Puerto Rico,Caguas,Caguas,"San Juan-Carolina-Caguas, PR",2024-05-22,daily_42602_2023.csv,2023
156365,72,25,7,42602,1,18.198712,-66.052237,NAD83,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-03-07,Parts per billion,NaN,11,46.0,5.436364,19.3,9,18,599,Instrumental - Chemiluminescence Teledyne API ...,NaN,Luis A. Ferre Highway,Puerto Rico,Caguas,Caguas,"San Juan-Carolina-Caguas, PR",2024-05-22,daily_42602_2023.csv,2023
1934,4,19,1036,42602,1,32.194671,-111.033167,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2024-08-22,Parts per billion,NaN,24,100.0,11.366667,26.4,23,25,574,Instrumental - Chemiluminescence Thermo Electr...,Ted Walker Park,6751 north Casa Grande Hwy,Arizona,Pima,Tucson,"Tucson, AZ",2024-11-14,daily_42602_2024.csv,2024
132926,49,3,7001,42602,1,41.945874,-112.233973,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-03-25,Parts per billion,NaN,21,88.0,0.000000,0.0,0,0,599,Instrumental - Chemiluminescence Teledyne API ...,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,Not in a city,"Ogden-Clearfield, UT",2024-05-22,daily_42602_2023.csv,2023
111088,44,7,40,42602,1,41.771987,-71.427881,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-04-20,Parts per billion,NaN,24,100.0,11.495833,23.7,6,22,574,Instrumental - Chemiluminescence Thermo Electr...,Near road (moved from Providence; Par/Hayes),Wellington Ave,Rhode Island,Providence,Cranston,"Providence-Warwick, RI-MA",2024-05-22,daily_42602_2023.csv,2023
76856,27,37,480,42602,1,44.706123,-93.285803,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,2023-06-11,Parts per billion,NaN,24,100.0,6.225000,21.0,21,20,99,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,Near Road I-35,16750 Kenyon Ave,Minnesota,Dakota,Lakeville,"Minneapolis-St. Paul-Bloomington, MN-WI",2024-05-22,daily_42602_2023.csv,2023


In [31]:
pd.concat([df_so2.head(3), df_so2.tail(3), df_so2.sample(5)])

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,file_name,file_year
0,1,73,23,42401,2,33.553056,-86.815000,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2022-01-01,Parts per billion,NaN,24,100.0,-0.191667,-0.1,12,0.0,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2023-04-18,daily_42401_2022.csv,2022
1,1,73,23,42401,2,33.553056,-86.815000,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2022-01-02,Parts per billion,NaN,24,100.0,-0.237500,-0.2,0,0.0,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2023-04-18,daily_42401_2022.csv,2022
2,1,73,23,42401,2,33.553056,-86.815000,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2022-01-03,Parts per billion,NaN,10,42.0,-0.380000,-0.2,0,0.0,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2023-04-18,daily_42401_2022.csv,2022
283300,72,123,4,42401,1,17.968352,-66.261365,WGS84,Sulfur dioxide,3-HR BLK AVG,SO2 3-hour 1971,2023-12-29,Parts per billion,NaN,8,100.0,1.300000,1.4,5,NaN,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,NaN,Rd # 3,Puerto Rico,Salinas,Salinas,"Coco, PR",2024-05-09,daily_42401_2023.csv,2023
283301,72,123,4,42401,1,17.968352,-66.261365,WGS84,Sulfur dioxide,3-HR BLK AVG,SO2 3-hour 1971,2023-12-30,Parts per billion,NaN,8,100.0,1.275000,1.9,2,NaN,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,NaN,Rd # 3,Puerto Rico,Salinas,Salinas,"Coco, PR",2024-05-09,daily_42401_2023.csv,2023
283302,72,123,4,42401,1,17.968352,-66.261365,WGS84,Sulfur dioxide,3-HR BLK AVG,SO2 3-hour 1971,2023-12-31,Parts per billion,NaN,8,100.0,1.125000,1.2,20,NaN,100,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,NaN,Rd # 3,Puerto Rico,Salinas,Salinas,"Coco, PR",2024-05-09,daily_42401_2023.csv,2023
11063,6,1,11,42401,1,37.814781,-122.282347,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2022-12-14,Parts per billion,NaN,22,92.0,0.272727,0.7,5,0.0,60,INSTRUMENTAL - PULSED FLUORESCENT,Oakland West,1100 21st Street,California,Alameda,Oakland,"San Francisco-Oakland-Hayward, CA",2023-03-28,daily_42401_2022.csv,2022
20784,6,65,8001,42401,9,33.999580,-117.416010,WGS84,Sulfur dioxide,3-HR BLK AVG,SO2 3-hour 1971,2023-09-04,Parts per billion,NaN,8,100.0,0.112500,0.2,23,NaN,560,INSTRUMENTAL - Pulsed Fluorescent 43C-TLE/43i-TLE,Rubidoux,"5888 MISSION BLVD., RUBIDOUX",California,Riverside,Rubidoux,"Riverside-San Bernardino-Ontario, CA",2024-05-09,daily_42401_2023.csv,2023
20925,6,67,6,42401,1,38.613779,-121.368014,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2023-01-23,Parts per billion,NaN,22,92.0,-0.186364,-0.1,18,0.0,600,Instrumental - Ultraviolet Fluorescence API 10...,Sacramento-Del Paso Manor,"DEL PASO-2701 AVALON DR, SACRAMENTO",California,Sacramento,Arden-Arcade,"Sacramento--Roseville--Arden-Arcade, CA",2024-04-22,daily_42401_2023.csv,2023
176205,38,105,105,42401,1,48.392666,-102.910693,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2023-02-16,Parts per billion,Included,24,100.0,0.375000,2.0,0,3.0,560,INSTRUMENTAL - Pulsed Fluorescent 43C-TLE/43i-TLE,NaN,AMERADA HESS #3,North Dakota,Williams,Not in a city,"Williston, ND",2024-03-14,daily_42401_2023.csv,2023


### Info

In [32]:
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1014725 entries, 0 to 388237
Data columns (total 31 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   State Code           1014725 non-null  int64  
 1   County Code          1014725 non-null  int64  
 2   Site Num             1014725 non-null  int64  
 3   Parameter Code       1014725 non-null  int64  
 4   POC                  1014725 non-null  int64  
 5   Latitude             1014725 non-null  float64
 6   Longitude            1014725 non-null  float64
 7   Datum                1014725 non-null  object 
 8   Parameter Name       1014725 non-null  object 
 9   Sample Duration      1014725 non-null  object 
 10  Pollutant Standard   1014725 non-null  object 
 11  Date Local           1014725 non-null  object 
 12  Units of Measure     1014725 non-null  object 
 13  Event Type           27286 non-null    object 
 14  Observation Count    1014725 non-null  int64  
 15  Obse

In [33]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 739126 entries, 0 to 283302
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State Code           739126 non-null  int64  
 1   County Code          739126 non-null  int64  
 2   Site Num             739126 non-null  int64  
 3   Parameter Code       739126 non-null  int64  
 4   POC                  739126 non-null  int64  
 5   Latitude             739126 non-null  float64
 6   Longitude            739126 non-null  float64
 7   Datum                739126 non-null  object 
 8   Parameter Name       739126 non-null  object 
 9   Sample Duration      739126 non-null  object 
 10  Pollutant Standard   739126 non-null  object 
 11  Date Local           739126 non-null  object 
 12  Units of Measure     739126 non-null  object 
 13  Event Type           19363 non-null   object 
 14  Observation Count    739126 non-null  int64  
 15  Observation Percent  7

In [34]:
df_no2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401419 entries, 0 to 156365
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State Code           401419 non-null  int64  
 1   County Code          401419 non-null  int64  
 2   Site Num             401419 non-null  int64  
 3   Parameter Code       401419 non-null  int64  
 4   POC                  401419 non-null  int64  
 5   Latitude             401419 non-null  float64
 6   Longitude            401419 non-null  float64
 7   Datum                401419 non-null  object 
 8   Parameter Name       401419 non-null  object 
 9   Sample Duration      401419 non-null  object 
 10  Pollutant Standard   401419 non-null  object 
 11  Date Local           401419 non-null  object 
 12  Units of Measure     401419 non-null  object 
 13  Event Type           12275 non-null   object 
 14  Observation Count    401419 non-null  int64  
 15  Observation Percent  4

In [35]:
df_co.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419198 entries, 0 to 89792
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State Code           419198 non-null  int64  
 1   County Code          419198 non-null  int64  
 2   Site Num             419198 non-null  int64  
 3   Parameter Code       419198 non-null  int64  
 4   POC                  419198 non-null  int64  
 5   Latitude             419198 non-null  float64
 6   Longitude            419198 non-null  float64
 7   Datum                419198 non-null  object 
 8   Parameter Name       419198 non-null  object 
 9   Sample Duration      419198 non-null  object 
 10  Pollutant Standard   419198 non-null  object 
 11  Date Local           419198 non-null  object 
 12  Units of Measure     419198 non-null  object 
 13  Event Type           5954 non-null    object 
 14  Observation Count    419198 non-null  int64  
 15  Observation Percent  41

- Todos os arquivos apresentam as 30 colunas 
- Possuem as seguintes mesmas colunas
- As seguinte colunas tem valores nulos em todos os arquivos:
  - Event Type: Indicates whether data measured during exceptional events are included in the summary.
  - AQI: The Air Quality Index for the day for the pollutant, if applicable.
  - Local Site Name: The name of the site (if any) given by the State, local, or tribal air pollution control agency that operates it.
  - Address: The approximate street address of the monitoring site.
  - CBSA Name: The name of the core bases statistical area (metropolitan area) where the monitoring site is located.

A coluna "Method Code" apresenta valores nulos apenas nos arquivos Ozonio e Co, e conforme a documentação:
- Method Code:
An internal system code indicating the method (processes, equipment, and protocols) used in gathering and measuring the sample. The method name is in the next column.

Verificado que em alguns casos o Method Name é preenchido enquanto a Method Code está NaN (nulo). Necessário uma análise para entender estes registros.

In [36]:
df_ozone[df_ozone['Method Code'].isnull()][['Method Code', 'Method Name']].sample()

,Method Code,Method Name
214273,NaN,-


In [37]:
df_co[df_co['Method Code'].isnull()][['Method Code', 'Method Name']].sample()

,Method Code,Method Name
22206,NaN,-


Avaliado que os registros os quais a coluna Method Code está nula (NaN) o Method Name tem valores "-", o que representa que não há nome, então não há uma real necessidade de tratamento. Mas pode ser interessante substituir o "-" por NA para facilitar a visualização e tratamento futuro.

In [38]:
df_ozone['Method Name'] = df_ozone['Method Name'].str.strip().replace('-', pd.NA)

In [39]:
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1014725 entries, 0 to 388237
Data columns (total 31 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   State Code           1014725 non-null  int64  
 1   County Code          1014725 non-null  int64  
 2   Site Num             1014725 non-null  int64  
 3   Parameter Code       1014725 non-null  int64  
 4   POC                  1014725 non-null  int64  
 5   Latitude             1014725 non-null  float64
 6   Longitude            1014725 non-null  float64
 7   Datum                1014725 non-null  object 
 8   Parameter Name       1014725 non-null  object 
 9   Sample Duration      1014725 non-null  object 
 10  Pollutant Standard   1014725 non-null  object 
 11  Date Local           1014725 non-null  object 
 12  Units of Measure     1014725 non-null  object 
 13  Event Type           27286 non-null    object 
 14  Observation Count    1014725 non-null  int64  
 15  Obse

In [40]:
df_ozone[(df_ozone['Method Code'].isnull())][['Method Code', 'Method Name']].sample()

,Method Code,Method Name
350041,NaN,<NA>


In [41]:
# Replicando para o dataframe de CO
df_co['Method Name'] = df_co['Method Name'].str.strip().replace('-', pd.NA)
df_co[(df_co['Method Code'].isnull())][['Method Code', 'Method Name']].sample()

,Method Code,Method Name
22206,NaN,<NA>


### Validação do ano do arquivo vs ano da coluna Date Local

In [42]:
df_all[['Date Local']].head()

,Date Local
0,2024-02-29
1,2024-03-01
2,2024-03-02
3,2024-03-03
4,2024-03-04


In [43]:
df_all[['file_year']].head()

KeyError: "None of [Index(['file_year'], dtype='object')] are in the [columns]"

In [22]:
df_all[df_all['Date Local'].str[:4] != df_all['file_year']].sample(10)

KeyError: 'file_year'

In [ ]:
df_all[df_all['Date Local'].str.split('-')[0] != df_all['file_year']].head(10)

### Reordenação das colunas

É mais fácil de analisar a olho humano dados e colunas que sejam nome representativos, do que IDS, então para isso, é importante reordenar as colunas do conjunto de dados para que façam sentido com as análise pensadas.

Exemplo: ao invés de "State Code" como 1ª coluna e o "State Name" como 24ª, é melhor ordernar de forma que uma esteja próxima a outra, ou até mesmo que o Nome venha antes da coluna de ID

In [20]:
# Listando as colunas
list(df_ozone.columns)

['State Code',
 'County Code',
 'Site Num',
 'Parameter Code',
 'POC',
 'Latitude',
 'Longitude',
 'Datum',
 'Parameter Name',
 'Sample Duration',
 'Pollutant Standard',
 'Date Local',
 'Units of Measure',
 'Event Type',
 'Observation Count',
 'Observation Percent',
 'Arithmetic Mean',
 '1st Max Value',
 '1st Max Hour',
 'AQI',
 'Method Code',
 'Method Name',
 'Local Site Name',
 'Address',
 'State Name',
 'County Name',
 'City Name',
 'CBSA Name',
 'Date of Last Change',
 'file_name',
 'file_year']

Como as colunas seguem um padrão de nomenclatura, poderiamos apenas fazer um sort delas, para ordenar por nome, isso já faria com que colunas de mesmo contexto estivessem lado a lado

In [21]:
sorted(list(df_ozone))

['1st Max Hour',
 '1st Max Value',
 'AQI',
 'Address',
 'Arithmetic Mean',
 'CBSA Name',
 'City Name',
 'County Code',
 'County Name',
 'Date Local',
 'Date of Last Change',
 'Datum',
 'Event Type',
 'Latitude',
 'Local Site Name',
 'Longitude',
 'Method Code',
 'Method Name',
 'Observation Count',
 'Observation Percent',
 'POC',
 'Parameter Code',
 'Parameter Name',
 'Pollutant Standard',
 'Sample Duration',
 'Site Num',
 'State Code',
 'State Name',
 'Units of Measure',
 'file_name',
 'file_year']

In [33]:
df_ozone.head(2)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,file_name,file_year
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-02-29,Parts per million,NaN,2,12.0,0.032000,0.032,23,30.0,47.0,INSTRUMENTAL - ULTRA VIOLET,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31,daily_44201_2024.csv,2024
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,Ozone 8-hour 2015,2024-03-01,Parts per million,NaN,34,200.0,0.024706,0.030,12,28.0,47.0,INSTRUMENTAL - ULTRA VIOLET,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31,daily_44201_2024.csv,2024


In [ ]:
columns_reorder = [
    'Parameter Name',

    'Date Local',
    'Date of Last Change',

    'CBSA Name',

    'County Name',
    'State Name',
    'City Name',
    'Address',
    

    'Observation Count',
    'Observation Percent',

    '1st Max Hour',
    '1st Max Value',
    'AQI',
    
    'Arithmetic Mean',
    
    'Event Type',

    'Local Site Name',
    
    'Method Code',
    'Method Name',
    
    'POC',
    
    'Pollutant Standard',
    'Sample Duration',
    
    
    'Units of Measure',

    'Latitude',
    'Longitude',
    'Datum',

    'Parameter Code',
    'County Code',
    'State Code',
    'Site Num',
    
]

df_ozone[columns_reorder].head(2)

,Parameter Name,Date Local,Date of Last Change,County Name,State Name,City Name,Address,Observation Count,Observation Percent,1st Max Hour,1st Max Value,AQI,Arithmetic Mean,CBSA Name,Event Type,Local Site Name,Method Code,Method Name,POC,Pollutant Standard,Sample Duration,Site Num,Units of Measure,Latitude,Longitude,Datum,Parameter Code,County Code,State Code,file_name,file_year
0,Ozone,2024-02-29,2024-10-31,Baldwin,Alabama,Fairhope,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",2,12.0,23,0.032,30.0,0.032000,"Daphne-Fairhope-Foley, AL",NaN,"FAIRHOPE, Alabama",47.0,INSTRUMENTAL - ULTRA VIOLET,1,Ozone 8-hour 2015,8-HR RUN AVG BEGIN HOUR,10,Parts per million,30.497478,-87.880258,NAD83,44201,3,1,daily_44201_2024.csv,2024
1,Ozone,2024-03-01,2024-10-31,Baldwin,Alabama,Fairhope,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",34,200.0,12,0.030,28.0,0.024706,"Daphne-Fairhope-Foley, AL",NaN,"FAIRHOPE, Alabama",47.0,INSTRUMENTAL - ULTRA VIOLET,1,Ozone 8-hour 2015,8-HR RUN AVG BEGIN HOUR,10,Parts per million,30.497478,-87.880258,NAD83,44201,3,1,daily_44201_2024.csv,2024


## Describe

In [14]:
df_ozone.describe().T

,count,mean,std,min,25%,50%,75%,max
State Code,1014725.0,26.683371,16.672032,1.000000,11.000000,26.000000,40.000000,80.000000
County Code,1014725.0,75.993697,81.144531,1.000000,21.000000,57.000000,103.000000,800.000000
Site Num,1014725.0,1325.524698,2647.597703,1.000000,6.000000,24.000000,1015.000000,9997.000000
Parameter Code,1014725.0,44201.000000,0.000000,44201.000000,44201.000000,44201.000000,44201.000000,44201.000000
POC,1014725.0,1.088073,0.349912,1.000000,1.000000,1.000000,1.000000,6.000000
Latitude,1014725.0,37.656293,4.903941,18.216038,34.144350,38.473670,40.961017,64.845690
Longitude,1014725.0,-95.128626,16.074139,-158.088613,-109.892490,-91.213556,-82.100646,-66.150615
Observation Count,1014725.0,22.435996,22.068947,1.000000,17.000000,17.000000,17.000000,408.000000
Observation Percent,1014725.0,132.003040,129.811574,6.000000,100.000000,100.000000,100.000000,2400.000000
Arithmetic Mean,1014725.0,0.033452,0.010806,-0.001000,0.025941,0.033471,0.040765,0.127214


In [15]:
df_no2.describe().T

,count,mean,std,min,25%,50%,75%,max
State Code,401419.0,28.326925,17.659851,1.000000,8.000000,29.000000,48.000000,72.000000
County Code,401419.0,79.972002,102.771312,1.000000,21.000000,47.000000,99.000000,760.000000
Site Num,401419.0,968.838518,1845.231262,1.000000,10.000000,48.000000,1025.000000,9997.000000
Parameter Code,401419.0,42602.000000,0.000000,42602.000000,42602.000000,42602.000000,42602.000000,42602.000000
POC,401419.0,1.229568,0.671191,1.000000,1.000000,1.000000,1.000000,6.000000
Latitude,401419.0,37.522496,4.826234,18.198712,33.999580,37.960400,40.816000,48.641930
Longitude,401419.0,-98.508234,16.602713,-159.366240,-112.355780,-97.337917,-83.000138,-66.052237
Observation Count,401419.0,23.004656,2.170395,1.000000,23.000000,24.000000,24.000000,24.000000
Observation Percent,401419.0,95.937280,9.017454,4.000000,96.000000,100.000000,100.000000,100.000000
Arithmetic Mean,401419.0,7.641672,6.693276,-3.333333,2.708333,5.637500,10.720833,64.058333


In [16]:
df_so2.describe().T

,count,mean,std,min,25%,50%,75%,max
State Code,739126.0,30.586380,15.689008,1.000000,17.000000,32.000000,42.000000,72.000000
County Code,739126.0,84.209959,92.196209,1.000000,21.000000,59.000000,115.000000,710.000000
Site Num,739126.0,931.563298,2052.569110,1.000000,7.000000,28.000000,1006.000000,9997.000000
Parameter Code,739126.0,42401.000000,0.000000,42401.000000,42401.000000,42401.000000,42401.000000,42401.000000
POC,739126.0,1.235374,0.793345,1.000000,1.000000,1.000000,1.000000,9.000000
Latitude,739126.0,37.833852,6.253036,17.967309,34.725352,38.956490,41.584957,64.845690
Longitude,739126.0,-92.828184,18.151723,-159.366240,-99.763424,-87.993440,-81.130222,-66.141683
Observation Count,739126.0,16.043300,9.733211,1.000000,8.000000,21.000000,24.000000,200.000000
Observation Percent,739126.0,104.095022,81.638126,4.000000,92.000000,100.000000,100.000000,2500.000000
Arithmetic Mean,739126.0,0.966794,8.639571,-3.916667,0.071429,0.372727,0.916667,965.377778


In [17]:
df_co.describe().T

,count,mean,std,min,25%,50%,75%,max
State Code,419198.0,25.925417,17.509172,1.000000,6.000000,26.000000,40.000000,72.000000
County Code,419198.0,72.510799,105.218798,1.000000,19.000000,39.000000,85.000000,760.000000
Site Num,419198.0,965.722580,1828.583039,1.000000,16.000000,75.000000,1028.000000,9997.000000
Parameter Code,419198.0,42101.000000,0.000000,42101.000000,42101.000000,42101.000000,42101.000000,42101.000000
POC,419198.0,1.206509,0.883934,1.000000,1.000000,1.000000,1.000000,9.000000
Latitude,419198.0,37.761738,5.551318,18.009558,34.068120,38.193600,41.182227,64.845690
Longitude,419198.0,-97.538494,19.056219,-158.088613,-116.347853,-94.635605,-80.256944,-66.052237
Observation Count,419198.0,25.032553,15.863165,1.000000,24.000000,24.000000,24.000000,504.000000
Observation Percent,419198.0,104.347120,66.087510,4.000000,100.000000,100.000000,100.000000,2100.000000
Arithmetic Mean,419198.0,0.268105,0.168887,-0.400000,0.161905,0.233333,0.345833,4.375000
